# Bench Test Analysis Notebook v0

Inputs:
- verification/templates/minimal_hardware_trials.csv (or filled trial log)
- verification/reports/benchmark_comparison.csv (optional model compare)

Outputs:
- results/plots/*.png
- verification/reports/bench_summary.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

ROOT = Path('..').resolve().parents[1]
DATA = ROOT / 'verification' / 'templates' / 'minimal_hardware_trials.csv'
OUT_PLOTS = ROOT / 'results' / 'plots'
OUT_PLOTS.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(DATA)
df.head()

In [ ]:
# Example derived metrics
if 'cycle_index' in df.columns and 'F_peak_N' in df.columns:
    df['f_peak_ratio'] = df.groupby(['soil_id','strategy_id','preload_N'])['F_peak_N'].transform(lambda x: x / x.iloc[0] if len(x) else x)

if {'strategy_id', 'preload_N', 'k_t_N_per_mm'}.issubset(df.columns):
    plt.figure(figsize=(7,4))
    sns.lineplot(data=df, x='preload_N', y='k_t_N_per_mm', hue='strategy_id', style='soil_id', marker='o')
    plt.title('Initial Stiffness vs Preload')
    plt.tight_layout()
    plt.savefig(OUT_PLOTS / 'stiffness_vs_preload.png', dpi=150)
    plt.close()

In [ ]:
# Aggregate summary
summary_cols = [c for c in ['F_peak_N','k_t_N_per_mm','x_slip_mm','E_diss_Nmm'] if c in df.columns]
if summary_cols:
    summary = df.groupby(['soil_id','strategy_id','preload_N'])[summary_cols].agg(['mean','std'])
    out_csv = ROOT / 'verification' / 'reports' / 'bench_summary.csv'
    summary.to_csv(out_csv)
    print(f'Wrote {out_csv}')
summary.head() if summary_cols else None